# Tutorial for selectionfunctions python module

In [ ]:
# Imports custom to how you run your notebook.
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
%config InlineBackend.figure_format = 'retina'
mpl.rcParams['figure.dpi']= 150

## Downloading the data files

In [ ]:
# Set the directory to where you want to store the large data files
from selectionfunctions.config import config
config['data_dir'] = '/Users/douglasboubert/Science/selectionfunctions/selectionfunctions/data'

In [ ]:
import selectionfunctions.boubert_everall_2019
selectionfunctions.boubert_everall_2019.fetch()

## Simple query

In [ ]:
from selectionfunctions.boubert_everall_2019 import BoubertEverall2019Query
from selectionfunctions.source_base import Source

boubert_everall_2019 = BoubertEverall2019Query()

In [ ]:
c = Source(
        '05h00m00.00000s',
        '+30d00m00.0000s',
        photometry={'gaia_g':21.2},
        frame='icrs')
print(boubert_everall_2019(c))

## Large query to create a map

In [ ]:
import PIL.Image
import astropy.units as u

def numpy2pil(a, vmin, vmax):
    a = np.clip((a - vmin) / (vmax - vmin), 0., 1.)
    a = (254.99 * a).astype('u1')
    return PIL.Image.fromarray(a)

w,h = (2056,1024)
l_0 = 130.

# Set up Bayestar query object
print('Loading Boubert & Everall 2019 selection function...')
boubert_everall_2019 = BoubertEverall2019Query()

# Create a grid of coordinates
print('Creating grid of coordinates...')
l = np.linspace(-180.+l_0, 180.+l_0, 2*w)
b = np.linspace(-90., 90., 2*h+2)
b = b[1:-1]
l,b = np.meshgrid(l, b)

l += (np.random.random(l.shape) - 0.5) * 360./(2.*w)
b += (np.random.random(l.shape) - 0.5) * 180./(2.*h)

sf = np.empty(l.shape+(3,), dtype='f8')

for k,G in enumerate([21.0, 21.2, 21.4]):
    # d = 5.    # We'll query integrated reddening to a distance of 5 kpc
    sources = Source(l*u.deg, b*u.deg, photometry={'gaia_g':G*np.ones(l.shape)}, frame='galactic')

    # Get the dust median reddening at each coordinate
    print('Querying map...')
    sf[:,:,k] = boubert_everall_2019.query(sources)

# Convert the output array to a PIL image and save
print('Saving image...')
img = numpy2pil(sf[::-1,::-1,:], 0., 1)
img = img.resize((w,h), resample=PIL.Image.LANCZOS)

plt.imshow(img)